In [1]:
#Код представляет собой реализацию дерева решений для классификации методом обучения с учителем. 

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits

In [2]:


# Класс для дерева решений
class DecisionTree:
    def __init__(self, maxDeep=10, minData=20, minCriterion=0.05, typeCriterion='Entropy', threshold=[0, 16], stepThreshold=0.5, isValid=False, splitVal=0.2, countVal=25):
        # Инициализация параметров дерева
        self.__maxDeep = maxDeep #Максимальная глубина дерева.
        self.__minData = minData #Минимальное количество данных в узле.
        self.__minCriterion = minCriterion #Минимальное значение критерия. 
        self.__typeCriterion = typeCriterion #Тип критерия.
        self.__threshold = threshold #Пределы порога для разделения данных. 
        self.__stepThreshold = stepThreshold # Шаг порога.
        self.__isValid = isValid #Флаг для использования кросс-валидации при обучении модели.
        self.__splitValid = splitVal #Доля данных, которые будут использоваться для валидации при кросс-валидации.
        self.__countVal = countVal # Количество итераций при кросс-валидации.
        self.__node = None #Узел дерева.
    # Получение параметров
    def get(self):
        return {'Max Deep': self.__maxDeep, 'Min Data': self.__minData, 'Min Criterion': self.__minCriterion, 'Type Criterion': self.__typeCriterion}

    # Рассчет значения критерия (Энтропия, Джини или Ошибка)
    def __Criterion(self, t_set):
        t_set = np.array(t_set)
        H = 0
        N_i = len(t_set)
        if N_i == 0:
            return 0
        if self.__typeCriterion == 'Entropy':
            for i in range(K):
                N_ik = len(t_set[t_set == i])
                if N_ik == 0:
                    continue
                H += (N_ik / N_i) * np.log(N_ik / N_i)
            return H * (-1)
        elif self.__typeCriterion == 'Gini':
            for i in range(K):
                N_ik = len(t_set[t_set == i])
                H += (N_ik / N_i) ** 2
            return 1 - H
        elif self.__typeCriterion == 'Error':
            MaxN_ik = 0
            for i in range(K):
                N_ik = len(t_set[t_set == i])
                if N_ik > MaxN_ik:
                    MaxN_ik = N_ik
            return 1 - (MaxN_ik / N_i)
    
    # Создание конечного узла (листа)
    def __CreateTerminalNode(self, t_set):
        t = np.zeros(K)
        N_i = len(t_set)
        for i in range(K):
            t[i] = len(t_set[t_set == i]) / N_i
        return {'Confidence': t, 'isTerminal': True}

    # Создание внутреннего узла
    def __CreateSplitNode(self, params):
        return {'Index_Cord': params['Index'], 'Threshold': params['Threshold'], 'isTerminal': False}
    
    # Получение параметров для разделения данных
    def __GetParams(self, x_set, t_set):
        threshold = 0
        maxI = -1
        left = list()
        right = list()
        coord_ind = 0
        coordcheck = np.arange(x_set.shape[1]) # Фичи
        np.random.shuffle(coordcheck)
        for j in range(x_set.shape[1]):
          for t in np.arange(self.__threshold[0], self.__threshold[1], self.__stepThreshold):
                left.clear()
                right.clear()

                for i in range(len(x_set)):
                    if x_set[i][coordcheck[j]] <= t:
                        left.append(t_set[i])
                    else:
                        right.append(t_set[i])

                I = self.__Criterion(t_set) - ((len(left) / len(t_set)) * self.__Criterion(left) +  (len(right) / len(t_set)) * self.__Criterion(right))
                if I > maxI: # Лучшие параметры
                    threshold = t
                    coord_ind = coordcheck[j]
                    maxI = I

        return {'Index': coord_ind, 'Threshold': threshold}

    # Разделение данных
    def __SplitData(self, x_set, params):
        Index = params['Index']
        t = params['Threshold']
        left = list()
        right = list()
        for i in range(len(x_set)):
            if x_set[i][Index] <= t:
                left.append(i)
            else:
                right.append(i)
        return left, right
    
    # Создание дерева
    def __CreateTree(self, x_set, t_set, deep):
        I = self.__Criterion(t_set)
        if deep > self.__maxDeep or len(x_set) < self.__minData or I < self.__minCriterion:
            node = self.__CreateTerminalNode(t_set)
            return node

        params = self.__GetParams(x_set, t_set)
        left_data, right_data = self.__SplitData(x_set, params)
        node = self.__CreateSplitNode(params)
        deep += 1
        node['left'] = self.__CreateTree(x_set[left_data], t_set[left_data], deep)
        node['right'] = self.__CreateTree(x_set[right_data], t_set[right_data], deep)

        return node

    # Обучение модели
    def fit(self, x_train, t_train):
        if self.__isValid:
            splitVal = int((1 - self.__splitValid) * x_train.shape[0])
            x_val, t_val = x_train[splitVal:], t_train[splitVal:]
            x_train, t_train = x_train[:splitVal], t_train[:splitVal]
            
            BestAccuracy = 0
            maxDeep = self.__maxDeep
            minData = self.__minData
            minCriterion = self.__minCriterion

            for i in range(self.__countVal):
                self.__maxDeep = np.random.randint(1, maxDeep)
                self.__minData = np.random.randint(1, minData)
                self.__minCriterion = np.random.randint(1, int(minCriterion * 100)) / 100
                self.__typeCriterion = np.random.choice(['Entropy', 'Gini', 'Error'])

                Tree = self.__CreateTree(x_train, t_train, 0)

                Acc = 0
                for k in range(len(x_val)):
                    if t_val[k] == np.argmax(self.predict(x_val[k], Tree)):
                        Acc += 1
                Acc /= len(x_val)

                if Acc > BestAccuracy:
                    BestAccuracy = Acc
                    BmaxDeep = self.__maxDeep
                    BminData = self.__minData
                    BminCriterion = self.__minCriterion
                    BestCriterion = self.__typeCriterion
                    BestNode = Tree
                
                print('Iter - {} | (Valid set) Accuracy - {}'.format(i + 1, Acc))
              
            self.__maxDeep = BmaxDeep
            self.__minData = BminData
            self.__minCriterion = BminCriterion
            self.__typeCriterion = BestCriterion
            self.__node = BestNode
        else:
            self.__node = self.__CreateTree(x_train, t_train, 0)
    
    # Предсказание
    def predict(self, X, Node=None):
        root = self.__node
        if Node != None:
          root = Node
        
        curNode = root
        if len(X.shape) == 1:
            while not curNode['isTerminal']:
                if X[curNode['Index_Cord']] <= curNode['Threshold']:
                    curNode = curNode['left']
                else:
                    curNode = curNode['right']
            return curNode['Confidence']
        else:
            pred = list()
            for i in range(X.shape[0]):
                curNode = root
                while not curNode['isTerminal']:
                    if X[i][curNode['Index_Cord']] <= curNode['Threshold']:
                        curNode = curNode['left']
                    else:
                        curNode = curNode['right']
                pred.append(curNode['Confidence'])
            return pred




In [3]:
# Функции
# Построение матрицы ошибок
def ConfusionMatrix(t_set, t_predict):
    SizeSet = len(t_set)
    Matrix = np.zeros((K, K))
    for i in range(SizeSet):
        Matrix[t_set[i]][int(t_predict[i])] += 1
    return Matrix

# Расчет точности
def Accuracy(Matrix):
    Acc = 0
    for i in range(len(Matrix)):
        Acc += Matrix[i][i]
    return Acc / np.sum(Matrix)






In [4]:
# Набор данных Digits
digits = load_digits()
N = len(digits.data)
D = len(digits.data[0])
K = len(digits.target_names)
x = np.copy(digits.data)
Target = digits.target
Images = digits.images

# Разделение данных
index_data = np.arange(N)
np.random.shuffle(index_data)
index_train = index_data[:int(N * 0.8)]
index_test = index_data[int(N * 0.8):]
x_train = x[index_train]
t_train = Target[index_train]
x_test = x[index_test]
t_test = Target[index_test]

In [17]:
# Параметры дерева решений
MaxDeep = 11 # Максимальная глубина
MinData = 30 # Минимальное количество данных в узле
MinCrit = 0.05 # Минимальное значение критерия
criterion = 'Gini' # Тип критерия: Энтропия, Джини, Ошибка
# Параметры разделения данных
MaxThr = 16 # Максимальный порог
StepThr = 0.5 # Шаг порога


In [18]:

# Обучение модели
model = DecisionTree(maxDeep=MaxDeep, minData=MinData, minCriterion=MinCrit, typeCriterion=criterion, threshold=[0, MaxThr], stepThreshold=StepThr, isValid=False)
model.fit(x_train, t_train)

# Классификация на обучающем и тестовом наборах
predTrain = model.predict(x_train)
predTest = model.predict(x_test)

predTrain = np.argmax(predTrain, axis=1)
predTest = np.argmax(predTest, axis=1)

In [19]:

# Метрики
print('(Обучающий набор) Матрица ошибок:')
ConfMatrixTrain = ConfusionMatrix(t_train, predTrain)
print(ConfMatrixTrain)
print('(Обучающий набор) Точность: {}'.format(Accuracy(ConfMatrixTrain)))

print('(Тестовый набор) Матрица ошибок:')
ConfMatrixTest = ConfusionMatrix(t_test, predTest)
print(ConfMatrixTest)
print('(Тестовый набор) Точность: {}'.format(Accuracy(ConfMatrixTest)))


(Обучающий набор) Матрица ошибок:
[[145.   1.   0.   0.   0.   0.   0.   2.   0.   0.]
 [  0. 137.   0.   2.   0.   0.   0.   0.   4.   3.]
 [  1.   4. 124.   1.   0.   3.   0.   2.   8.   5.]
 [  0.   2.   1. 128.   2.   1.   0.   4.   3.   3.]
 [  0.   3.   0.   0. 127.   1.   0.   6.   0.   0.]
 [  1.   4.   0.   1.   5. 137.   0.   0.   0.   1.]
 [  0.   6.   0.   1.   4.   4. 137.   2.   0.   0.]
 [  0.   0.   0.   3.   2.   2.   0. 126.   0.   1.]
 [  2.   6.   2.   1.   1.   1.   0.   4. 121.   0.]
 [  0.   5.   0.   2.   4.   2.   0.   5.   3. 118.]]
(Обучающий набор) Точность: 0.9046624913013221
(Тестовый набор) Матрица ошибок:
[[29.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0. 30.  1.  1.  1.  0.  0.  0.  1.  2.]
 [ 0.  2. 24.  0.  0.  0.  0.  1.  2.  0.]
 [ 0.  0.  3. 32.  0.  1.  0.  0.  0.  3.]
 [ 1.  2.  0.  0. 36.  1.  0.  3.  1.  0.]
 [ 1.  1.  0.  1.  1. 28.  0.  0.  1.  0.]
 [ 0.  1.  0.  0.  1.  0. 25.  0.  0.  0.]
 [ 0.  3.  0.  2.  1.  1.  0. 38.  0.  0.]
 [ 0.  2.  

: 